In [ ]:
import collections
import copy as cp
import math
import numpy as np
import pandas as pd
import scipy as sp

PI = math.pi

PITCH = 0.55  # 螺距 55cm
SPEED = 1  # 头节点速率 1m/s
INIT_ANGLE = 16 * 2 * PI  # 初始角位置

NUM_OF_BODY_NODES = 223  # 不包括头节点
HEAD_BENCH_LEN = 3.41  # 龙头长 341cm
BODY_BENCH_LEN = 2.20  # 龙身板凳长 220cm
END_LEN = 0.275  # 孔中心距离近端 27.5cm

TOT_TIME = 300
TIMESTAMPS = [60 * i for i in range(6)]
KEY_NODES = [50 * i for i in range(5)] + [NUM_OF_BODY_NODES - 1]


def toCartesian(rad: float, ang: float) -> tuple[float, float]:
    return (rad * math.cos(ang), rad * math.sin(ang))


def toPolar(x: float, y: float) -> tuple[float, float]:
    return (math.hypot(x, y), math.atan2(y, x))


# 等距螺线表达式
def radialOf(ang: float) -> float:
    return PITCH * ang / (2 * PI)


def normalize(vec) -> tuple[float, float]:
    length = math.hypot(*vec)
    return tuple(x / length for x in vec)


def tangentVectorAt(ang: float) -> tuple[float, float]:
    cos, sin = math.cos(ang), math.sin(ang)
    return (cos - ang * sin, sin + ang * cos)


# 单调递增的奇函数，非常漂亮
def arcLenOf(ang: float) -> float:
    return PITCH * (ang * math.sqrt(ang**2 + 1) + math.asinh(ang)) / (4 * PI)


TOT_ARC_LEN = arcLenOf(INIT_ANGLE)


# 求给定时刻头节点位置
def headNodeAngleAt(tm: float) -> float:
    eq = lambda theta: arcLenOf(theta) + SPEED * tm - TOT_ARC_LEN
    return sp.optimize.brentq(eq, 0.0, INIT_ANGLE, disp=True)


# 求给定时刻头节点速度
def headNodeVelocityAt(tm: float) -> tuple[float, float]:
    v = normalize(tangentVectorAt(headNodeAngleAt(tm)))
    return (v[0] * SPEED, v[1] * SPEED)


# 由当前节点的角位置求下一个节点的角位置
def angleOfNextNodeOf(cur: float, dist: float) -> float:
    eq = (
        lambda theta: PITCH**2
        * (theta**2 - 2 * cur * theta * math.cos(theta - cur) + cur**2)
        - 4 * PI**2 * dist**2
    )
    return sp.optimize.brentq(eq, cur, cur + PI, disp=True)


# 求给定时刻各节点角位置
def anglesOfNodesAt(tm: float) -> list[float]:
    head_ang = headNodeAngleAt(tm)
    angs = [head_ang, angleOfNextNodeOf(head_ang, HEAD_BENCH_LEN - 2 * END_LEN)]
    for i in range(1, NUM_OF_BODY_NODES):
        nxt_ang = angleOfNextNodeOf(angs[-1], BODY_BENCH_LEN - 2 * END_LEN)
        angs.append(nxt_ang)
    return angs


# 由当前节点的角位置和速度求下一个节点的角位置和速度
def angleAndVelocityOfNextNodeOf(
    cur_ang: float, cur_vel: tuple[float, float], dist: float, eps: float = 1e-9
) -> tuple[float, tuple[float, float]]:
    nxt_ang = angleOfNextNodeOf(cur_ang, dist)

    cur_pos = toCartesian(radialOf(cur_ang), cur_ang)
    nxt_pos = toCartesian(radialOf(nxt_ang), nxt_ang)
    bench_unit_vec = normalize((nxt_pos[0] - cur_pos[0], nxt_pos[1] - cur_pos[1]))
    v_along_bench = np.dot(cur_vel, bench_unit_vec)

    tan_vec = tangentVectorAt(nxt_ang)

    # Cramer法则
    det = -np.dot(bench_unit_vec, tan_vec)
    det_x = -tan_vec[0] * v_along_bench
    det_y = -tan_vec[1] * v_along_bench
    if abs(det) < eps:
        raise ValueError(
            "Singular configuration: bench is perpendicular to tangent vector."
        )

    return (nxt_ang, (det_x / det, det_y / det))


# 求某时刻各节点的角位置和速度
def anglesAndVelocitiesOfNodesAt(
    tm: float,
) -> tuple[list[float], list[tuple[float, float]]]:
    head_ang, head_vel = headNodeAngleAt(tm), headNodeVelocityAt(tm)
    angs = [head_ang, angleOfNextNodeOf(head_ang, HEAD_BENCH_LEN - 2 * END_LEN)]
    vels = [
        head_vel,
        angleAndVelocityOfNextNodeOf(head_ang, head_vel, HEAD_BENCH_LEN - 2 * END_LEN)[
            1
        ],
    ]
    for i in range(1, NUM_OF_BODY_NODES):
        nxt_ang, nxt_vel = angleAndVelocityOfNextNodeOf(
            angs[-1], vels[-1], BODY_BENCH_LEN - 2 * END_LEN
        )
        angs.append(nxt_ang)
        vels.append(nxt_vel)
    return (angs, vels)


def solve():
    """
    求解各节点在[0, TOT_TIME]各时刻的直角坐标位置和速度，分别保存在两个CSV中。
    位置CSV文件的索引列均依次命名为
    head x(m), head y(m), node 0 x(m), node 0 y(m), node 1 x(m), node 1 y(m), ...。
    速度CSV的同上，惟单位改为m/s。
    每个CSV的列依次命名为Node, 0 s, 1 s, 2 s, ...。
    """

    # 1. 准备行索引名称
    pos_node_names = ["head x(m)", "head y(m)"]
    vel_node_names = ["head x(m/s)", "head y(m/s)"]
    for i in range(NUM_OF_BODY_NODES):
        pos_node_names.extend([f"node {i} x(m)", f"node {i} y(m)"])
        vel_node_names.extend([f"node {i} x(m/s)", f"node {i} y(m/s)"])

    pos_data = collections.OrderedDict()
    vel_data = collections.OrderedDict()

    for t in range(TOT_TIME + 1):
        angs, vels = anglesAndVelocitiesOfNodesAt(float(t))

        pos_data[f"{t} s"] = np.array(
            [toCartesian(radialOf(ang), ang) for ang in angs]
        ).flatten()
        vel_data[f"{t} s"] = np.array(vels).flatten()

    df_pos = pd.DataFrame(pos_data, index=pos_node_names)
    df_pos.reset_index(inplace=True)
    df_pos.rename(columns={"index": "Node"}, inplace=True)
    df_pos.to_csv("results1_positions.csv", index=False, float_format="%.06f")

    df_vel = pd.DataFrame(vel_data, index=vel_node_names)
    df_vel.reset_index(inplace=True)
    df_vel.rename(columns={"index": "Node"}, inplace=True)
    df_vel.to_csv("results1_velocities.csv", index=False, float_format="%.06f")


solve()